# Preparation

In [34]:
import time
from preprocess import *


# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 2

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

Saving vectors of label - 'cipi': 100%|█████████████████████████████████████████████| 142/142 [00:00<00:00, 217.46it/s]


In [22]:
X_train.shape

(177, 20, 11, 1)

In [33]:
X_test.shape

(119, 20, 11, 1)

# TORCH MODEL

In [42]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(channel, 32, kernel_size=2)
        self.conv2 = nn.Conv2d(32, 48, kernel_size=2)
        self.conv3 = nn.Conv2d(48, 120, kernel_size=2)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout(p=0.25)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(120*6*6, 128)
        self.dropout2 = nn.Dropout(p=0.25)
        self.fc2 = nn.Linear(128, 64)
        self.dropout3 = nn.Dropout(p=0.4)
        self.fc3 = nn.Linear(64, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.relu(self.conv3(x))
        x = self.maxpool(x)
        x = self.dropout1(x)
        x = self.flatten(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout2(x)
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout3(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
        
model = Model()

# DATALOADER

In [24]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = torch.from_numpy(self.data[index])
        y = torch.from_numpy(self.labels[index])
        return x, y

# create a CustomDataset object for the training data
train_dataset = CustomDataset(X_train, y_train_hot)

# create a DataLoader object for the training data
batch_size = 10
shuffle = True
num_workers = 0
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

In [25]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [28]:
for i, (inputs, labels) in enumerate(train_loader):
    print(i, inputs.shape)

0 torch.Size([10, 20, 11, 1])
1 torch.Size([10, 20, 11, 1])
2 torch.Size([10, 20, 11, 1])
3 torch.Size([10, 20, 11, 1])
4 torch.Size([10, 20, 11, 1])
5 torch.Size([10, 20, 11, 1])
6 torch.Size([10, 20, 11, 1])
7 torch.Size([10, 20, 11, 1])
8 torch.Size([10, 20, 11, 1])
9 torch.Size([10, 20, 11, 1])
10 torch.Size([10, 20, 11, 1])
11 torch.Size([10, 20, 11, 1])
12 torch.Size([10, 20, 11, 1])
13 torch.Size([10, 20, 11, 1])
14 torch.Size([10, 20, 11, 1])
15 torch.Size([10, 20, 11, 1])
16 torch.Size([10, 20, 11, 1])
17 torch.Size([7, 20, 11, 1])


torch.Size([10, 20, 11, 1])


RuntimeError: Given groups=1, weight of size [32, 1, 2, 2], expected input[10, 20, 11, 1] to have 1 channels, but got 20 channels instead

In [26]:
for i, (inputs, labels) in enumerate(train_loader):
    # Perform a forward pass through the model
    outputs = model(inputs)

    # Compute the loss
    loss = my_loss_function(outputs, labels)

    # Backpropagate the gradients and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print some information every few batches
    if i % 10 == 0:
        print(f'Batch {i}: Loss = {loss.item():.4f}')

RuntimeError: Given groups=1, weight of size [32, 1, 2, 2], expected input[10, 20, 11, 1] to have 1 channels, but got 20 channels instead

In [43]:
x = torch.randn(32, 1, 2, 2)
print(x.shape)
output = model(x)

torch.Size([32, 1, 2, 2])


RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (2 x 2). Kernel size can't be greater than actual input size

In [45]:
model(x)

RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (2 x 2). Kernel size can't be greater than actual input size